# 18650 Battery Models

## Aryan, Avery, Cormac, & Tyler

In [4]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm

In [5]:
regular = pd.read_csv('.\\battery_alt_dataset\\regular_alt_batteries.csv')

C:\Users\tyler\AppData\Local\Temp\ipykernel_14540\1882604773.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  regular = pd.read_csv('.\\battery_alt_dataset\\regular_alt_batteries.csv')


In [17]:
def battLabeller(df):
    '''
    df - battery dataset
    feed battery dataset, add column to denote discharge phase
    '''
    label = 1
    df['dischargePhase'] = 0
    # for time in range(len(df['start_time'])-1):
    for time in tqdm(range(len(df['start_time'])-1), total=len(df['start_time'])-1):
        if df['mode'][time] == -1:
            df.at[time, 'dischargePhase'] = label
            if df['mode'][time+1] == 0 or df['mode'][time+1] == 1:
                label += 1
    return df

In [32]:
regular = regular[regular['mode'] != 0]
regular = regular[regular['mode'] != 1]

In [40]:
regular

,start_time,time,mode,voltage_charger,temperature_battery,voltage_load,current_load,temperature_mosfet,temperature_resistor,mission_type,dischargePhase,startTime,finishTime
324,2022-07-19 11:10:00,300.523,-1.0,8.340,22.963,-0.026,0.152000,22.64,23.20,0.0,1.0,0,0
325,2022-07-19 11:10:00,301.463,-1.0,8.041,23.155,8.329,2.523000,22.65,23.21,0.0,1.0,0,0
326,2022-07-19 11:10:00,302.404,-1.0,8.030,23.17,8.320,2.521000,22.66,23.20,0.0,1.0,0,0
327,2022-07-19 11:10:00,303.345,-1.0,8.023,23.186,8.312,2.523000,22.68,23.21,0.0,1.0,0,0
328,2022-07-19 11:10:00,304.288,-1.0,8.016,23.201,8.305,2.521000,22.71,23.21,0.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28357171,2022-10-16 12:55:00,1506502.662,-1.0,6.361,41.868,6.356,14.816191,31.22,27.72,1.0,4896.0,0,0
28357172,2022-10-16 12:55:00,1506503.726,-1.0,6.392,42.073,6.409,14.850649,31.24,27.77,1.0,4896.0,0,0
28357173,2022-10-16 12:55:00,1506504.785,-1.0,6.365,42.259,6.379,14.848449,31.28,27.81,1.0,4896.0,0,0
28357174,2022-10-16 12:55:00,1506505.841,-1.0,6.382,42.437,6.377,14.819122,31.31,27.86,1.0,4896.0,0,0


In [66]:
def timeFinder(df):
    '''
    df - dataframe
    takes dataframe and finds time from start of phase to end of phase
    '''
    df['startTime'] = 0
    df['finishTime'] = 0
    #for phase in df['dischargePhase'].unique():
    for phase in tqdm(df['dischargePhase'].unique(), total=len(df['dischargePhase'].unique())):
        currPhase = df[df['dischargePhase'] == phase]
        #print(currPhase)
        #print(phase)
        # Find start and end time
        start = currPhase['time'].iloc[0] 
        finish = currPhase['time'].iloc[-1]
        
        # Update start and end time
        df.loc[df['dischargePhase'] == phase, 'startTime'] = start
        df.loc[df['dischargePhase'] == phase, 'finishTime'] = finish
    return df

In [52]:
cols = ['voltage_charger','temperature_battery','current_load','dischargePhase','startTime','finishTime']
regular = regular[cols]
regular = regular.dropna(subset=['temperature_battery'])

In [63]:
regular.to_csv('regular.csv')

In [23]:
recommissioned = pd.read_csv('.\\battery_alt_dataset\\recommissioned_batteries.csv')
#regular = pd.read_csv('.\\battery_alt_dataset\\regular_alt_batteries.csv')
second = pd.read_csv('.\\battery_alt_dataset\\second_life_batteries.csv')

C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\2358409494.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  recommissioned = pd.read_csv('.\\battery_alt_dataset\\recommissioned_batteries.csv')


In [24]:
battLabeller(recommissioned)

  0%|          | 0/18051059 [00:00<?, ?it/s]

100%|██████████| 18051059/18051059 [03:24<00:00, 88245.35it/s] 


,start_time,time,mode,voltage_charger,temperature_battery,voltage_load,current_load,temperature_mosfet,temperature_resistor,mission_type,dischargePhase
0,2022-11-26 09:49:00,0.000,0.0,0.000,0.0,NaN,NaN,NaN,NaN,NaN,0
1,2022-11-26 09:49:00,0.349,0.0,0.000,0.0,NaN,NaN,NaN,NaN,NaN,0
2,2022-11-26 09:49:00,0.496,0.0,0.000,0.0,NaN,NaN,NaN,NaN,NaN,0
3,2022-11-26 09:49:00,0.935,0.0,8.365,22.611,NaN,NaN,NaN,NaN,NaN,0
4,2022-11-26 09:49:00,1.843,0.0,8.365,22.611,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
18051055,2022-11-22 14:36:00,1157842.807,0.0,-0.062,26.758,NaN,NaN,NaN,NaN,NaN,0
18051056,2022-11-22 14:36:00,1157843.820,0.0,-0.062,26.75,NaN,NaN,NaN,NaN,NaN,0
18051057,2022-11-22 14:36:00,1157844.834,0.0,-0.066,26.742,NaN,NaN,NaN,NaN,NaN,0
18051058,2022-11-22 14:36:00,1157845.846,0.0,-0.065,26.734,NaN,NaN,NaN,NaN,NaN,0


In [26]:
def process_battery_data(df):
    '''
    Takes in df of battery data and reduces each phase
    to a single row, averaging temp, voltage, and current
    and finding the time to death for each discharge phase
    
    Parameters:
    df (dataFrame): input df of battery data sent through
    battLabeller and timeFinder
    
    Returns:
    df (transformed): averaged values and new column
    end_time which is the difference of finishTime and startTime
    '''
    # Run battery df through functions time_end via difference
    df = battLabeller(df)
    df = df[df['mode'] != 0]
    df = df[df['mode'] != 1]
    df = timeFinder(df)
    cols = ['voltage_charger','temperature_battery','current_load','dischargePhase','startTime','finishTime']
    df = df[cols]
    df = df.dropna(subset=['dischargePhase'])
    
    df['time_end'] = (df['finishTime'] - df['startTime'])
    
    # Cast columns to floats
    numeric_cols = ['voltage_charger','temperature_battery','current_load']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col],errors = 'coerce')
    
    # Group by dischargePhase and aggregate other columns
    aggregated_df = df.groupby('dischargePhase').agg({
        'voltage_charger':'mean',
        'temperature_battery':'mean',
        'current_load':'mean',
        'time_end':'first'
    }).reset_index()
    aggregated_df['mode'] = 1
    
    return aggregated_df

In [45]:
second = pd.read_csv('.\\battery_alt_dataset\\second_life_batteries.csv')
second = battLabeller(second)
second = second[second['mode'] != 0]
second = second[second['mode'] != 1]

100%|██████████| 7119971/7119971 [01:39<00:00, 71312.74it/s] 


In [69]:
second = pd.read_csv('.\\battery_alt_dataset\\second_life_batteries.csv')
second = process_battery_data(second)

  0%|          | 0/1405 [00:00<?, ?it/s]C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\2932779929.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '600.096' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['dischargePhase'] == phase, 'startTime'] = start
C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\2932779929.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3205.154' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['dischargePhase'] == phase, 'finishTime'] = finish
100%|██████████| 1405/1405 [00:13<00:00, 107.01it/s]


In [76]:
regular = pd.read_csv('.\\data\\regular_alt_batteries.csv')
regular = process_battery_data(regular)

C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\1923300047.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  regular = pd.read_csv('.\\data\\regular_alt_batteries.csv')
  0%|          | 0/4896 [00:00<?, ?it/s]C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\2932779929.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '300.523' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['dischargePhase'] == phase, 'startTime'] = start
C:\Users\tyler\AppData\Local\Temp\ipykernel_12716\2932779929.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3803.755' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['dischargePhase'] == phase, 'finishTime'] = finish
100%|██████████| 489

In [77]:
regular.to_csv('regular.csv')

In [78]:
regular

,dischargePhase,voltage_charger,temperature_battery,current_load,time_end,mode
0,1,6.993291,25.916404,2.519990,3503.232,1
1,2,6.448755,64.757509,16.253802,538.454,1
2,3,6.474374,67.185977,16.246017,540.349,1
3,4,6.472878,67.642913,16.249888,539.218,1
4,5,6.353341,70.845676,17.602098,494.909,1
...,...,...,...,...,...,...
4891,4892,6.156449,62.168234,15.119306,477.210,1
4892,4893,6.148608,62.504518,15.121327,476.649,1
4893,4894,6.148097,62.199106,15.121059,475.834,1
4894,4895,6.163466,61.660934,15.120537,467.765,1
